In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
from cartopy.feature import ShapelyFeature
import PIL


In [9]:
def calculate_degrees(file_id):
    
    # Read in GOES ABI fixed grid projection variables and constants
    x_coordinate_1d = file_id.variables['x'][:]  # E/W scanning angle in radians
    y_coordinate_1d = file_id.variables['y'][:]  # N/S elevation angle in radians
    # projection_info = file_id.variables['goes_imager_projection']
    projection_info = file_id['goes_imager_projection']
    # dataset['goes_imager_projection'].longitude_of_projection_origin
    
    
    lon_origin = projection_info.longitude_of_projection_origin
    H = projection_info.perspective_point_height+projection_info.semi_major_axis
    r_eq = projection_info.semi_major_axis
    r_pol = projection_info.semi_minor_axis
    
    # Create 2D coordinate matrices from 1D coordinate vectors
    x_coordinate_2d, y_coordinate_2d = np.meshgrid(x_coordinate_1d, y_coordinate_1d)
    
    # Equations to calculate latitude and longitude
    lambda_0 = (lon_origin*np.pi)/180.0  
    a_var = np.power(np.sin(x_coordinate_2d),2.0) + (np.power(np.cos(x_coordinate_2d),2.0)*(np.power(np.cos(y_coordinate_2d),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(y_coordinate_2d),2.0))))
    b_var = -2.0*H*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    c_var = (H**2.0)-(r_eq**2.0)
    r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)
    s_x = r_s*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    s_y = - r_s*np.sin(x_coordinate_2d)
    s_z = r_s*np.cos(x_coordinate_2d)*np.sin(y_coordinate_2d)
    
    # Ignore numpy errors for sqrt of negative number; occurs for GOES-16 ABI CONUS sector data
    np.seterr(all='ignore')
    
    abi_lat = np.nan_to_num((180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y)))))))
    abi_lon = np.nan_to_num((lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi))
    
    return abi_lat, abi_lon

In [13]:
def view(xds):
    R = xds['Rad'].data
    dat = xds.metpy.parse_cf('Rad')
    geos = dat.metpy.cartopy_crs
    
    # We also need the x (north/south) and y (east/west) axis sweep of the ABI data
    x = dat.x
    y = dat.y

    # The geostationary projection is the easiest way to plot the image on a map. Essentially, we are stretching the image across a map with the same projection and dimensions as the data.
    fig = plt.figure(figsize=(15, 12))
    
    # Create axis with Geostationary projection
    ax = fig.add_subplot(1, 1, 1, projection=geos)
    
    # Add the RGB image to the figure. The data is in the same projection as the
    # axis we just created.
    ax.imshow(R, origin='upper',
              extent=(x.min(), x.max(), y.min(), y.max()), transform=geos)
    
    # Add Coastlines and States
    ax.coastlines(resolution='50m', color='black', linewidth=.25)
    ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)
    ax.add_feature(ccrs.cartopy.feature.OCEAN, zorder=1)
    

In [12]:
def locate_pixels(coords, q_lat, q_lon):
    query = np.array([q_lat, q_lon])
    diff = np.abs(coords-query).sum(axis=-1)
    return np.unravel_index(diff.argmin(), diff.shape)

In [8]:
def mask_image(xds):
    reader = shpreader.Reader("data/ne_50m_admin_0_countries/ne_50m_admin_0_countries.shp")
    kenya = [country for country in reader.records() if country.attributes["NAME_LONG"] == "United States"][0]
    shape_feature = ShapelyFeature([kenya.geometry], ccrs.PlateCarree(), facecolor="lime", edgecolor='black', lw=1)
    
    R = xds['Rad'].data
    dat = xds.metpy.parse_cf('Rad')
    geos = dat.metpy.cartopy_crs
    x = dat.x
    y = dat.y

    fig = plt.figure(figsize=(15, 12))
    ax = fig.add_subplot(1, 1, 1, projection=geos)
    ax.add_feature(shape_feature, color='white', zorder=1)
    ax.imshow(R, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos)
    
    fig.canvas.draw()
    temp_canvas = fig.canvas
    rgb_str = temp_canvas.tostring_rgb()
    plt.close()
    
    pil_image = PIL.Image.frombytes('RGB', temp_canvas.get_width_height(),  temp_canvas.tostring_rgb())
    arr = np.array(pil_image.convert('L'))
    arr[arr==255]=0
    # np.nonzero(arr)# np.unique(arr[...,-1])
    x, y = np.nonzero(arr)
    xl,xr = x.min(),x.max()
    yl,yr = y.min(),y.max()
    cropped = arr[xl:xr+1, yl:yr+1]

    R = xds['Rad'].data[:cropped.shape[0], :cropped.shape[1]]
    R[cropped!=0] = np.nan
    R[R==255] = np.nan

    vals, counts = np.unique(R[:R.shape[0]//10, :R.shape[1]//10], return_counts=True)
    replace = vals[np.argsort(counts)[-1]]
    # print('Most common value: ', replace)
    # # vals[0]
    # print('First row: ', R[0])
    # print('First col: ', R[:,0])
    if not np.isnan(replace):
        R[R==replace] = np.nan
    
    return R